In [23]:
# Test
import os
import numpy as np
import pandas as pd
import zipfile
import datetime
import datetime
from xgboost import XGBClassifier, XGBRegressor
import kaggle
import pickle
from tensorflow.keras.models import Model

import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

<module 'auto_co2' from 'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks\\auto_co2\\__init__.py'>

In [2]:
import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

<module 'auto_co2' from 'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks\\auto_co2\\__init__.py'>

In [3]:
# test ludo supr
%pwd

'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks'

In [19]:
# test ludo supr
# Cellule à tester
import sys
sys.path.append("../src/")
import auto_co2 as co2

In [17]:
auth_file_path = os.environ.get('KAGGLE_CONFIG_DIR')
print(auth_file_path)

None


In [8]:
dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
dataset_name = dataset_id.split('/')[-1]  # Get the dataset name
print(os.environ.get('KAGGLE_CONFIG_DIR'))
# Save the original KAGGLE_CONFIG_DIR
original_kaggle_config_dir = os.environ.get('KAGGLE_CONFIG_DIR')
# Point KAGGLE_CONFIG_DIR to the directory containing kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(auth_file_path)

os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')

    # Restore the original KAGGLE_CONFIG_DIR
if original_kaggle_config_dir is not None:
        os.environ['KAGGLE_CONFIG_DIR'] = original_kaggle_config_dir
else:
    del os.environ['KAGGLE_CONFIG_DIR']


None


NameError: name 'auth_file_path' is not defined

In [6]:
dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
filepath='../data/raw'
os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')

0

In [7]:
dataset_id

'matthieulenozach/automobile-co2-emissions-eu-2021'

In [8]:
dataset_name = dataset_id.split('/')[-1]

In [18]:
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data
df = load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw')

In [4]:
# Cellule qui marche chez moi. Les manip autour de 'KAGGLE_CONFIG_DIR' ne donnent rien 
# a retester ?
# Todo: Un test pour voir si ce dataset (meme id?, meme taille ?) a déja été 
# chargé car 4 min

# Modifiée
def download_co2_data(filepath='../data/raw'):
    dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
    os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
    dataset_name = dataset_id.split('/')[-1]
    return dataset_name

# C'est la meme
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data

# Modifiée
def download_and_load_co2_data(filepath='../data/raw'):
    dataset_name = download_co2_data(filepath)
    data = load_co2_data(dataset_name, filepath)
    return data

# df = download_and_load_co2_data(filepath='../data/raw')
df.to_csv('../data/raw/df_test.csv')

# time: 7 min

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [21]:
df

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,SBDTTCX0,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,DLAAX0AC4,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,YLDC1FX,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,DLAAX0AC4,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,R,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [3]:
# Fonctions à tester, qui traite les deux cas.

def download_co2_data(filepath='../data/raw', auth_file_path=None):
    dataset_id = 'matthieulenozach/automobile-co2-emissions-eu-2021'
    dataset_name = dataset_id.split('/')[-1]

    if auth_file_path is None:   
        os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
    else:
        # Save the original KAGGLE_CONFIG_DIR
        original_kaggle_config_dir = os.environ.get('KAGGLE_CONFIG_DIR')
        # Point KAGGLE_CONFIG_DIR to the directory containing kaggle.json
        os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(auth_file_path)

        os.system(f'kaggle datasets download -d {dataset_id} -p {filepath}')
        # Restore the original KAGGLE_CONFIG_DIR
        os.environ['KAGGLE_CONFIG_DIR'] = original_kaggle_config_dir
    return dataset_name

# non modifiee
def load_co2_data(dataset_name="automobile-co2-emissions-eu-2021", filepath='../data/raw'):
    # read zip file and extract in {filepath}
    with zipfile.ZipFile(f'{filepath}/{dataset_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(filepath)
    filename = zip_ref.namelist()[0] 
    data = pd.read_csv(f'{filepath}/{filename}', low_memory=False)
    return data

# Modifiée
def download_and_load_co2_data(filepath='../data/raw', auth_file_path=None):
    dataset_name = download_co2_data(filepath, auth_file_path)
    data = load_co2_data(dataset_name, filepath)
    return data

df = download_and_load_co2_data(filepath='../data/raw')
df.to_csv('../data/raw/df_test.csv')



In [4]:
df

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,SBDTTCX0,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,DLAAX0AC4,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,YLDC1FX,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,DLAAX0AC4,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,R,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


# New Test

In [2]:
# Test
import os
import numpy as np
import pandas as pd
import zipfile
import datetime
import datetime
from xgboost import XGBClassifier, XGBRegressor
import kaggle
import pickle
from tensorflow.keras.models import Model

import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

<module 'auto_co2' from 'd:\\0 PERSO\\00 LUDO\\Google Drive\\0 LUDO\\Projets\\Cours_info\\05 Projet Dts\\github\\car-co2-prediction_pre-release\\notebooks\\auto_co2\\__init__.py'>

In [37]:
### Modifications
# La fonction a été modifiée (relire ?) pour marcher chez toi et chez moi.
# Cette cellule est modifiée avec l'argument auth_file_path [Optionnel]

### Remarque : Il n'y a que la ligne 2 sur ton Nbook. Mais ca marche pas si on a pas déja chargé
# Donc dans ce cas executer la ligne 1. 
# TODO: Voir s'il faut mettre que la ligne 1, avec un test pour savoir si déja chargé (dans la fonction)

# df = co2.data.download_and_load_co2_data(filepath='../data/raw')
df = co2.data.load_co2_data()   # zip déja uploadé



In [56]:
# Reduction de dimmensions
# n = 100
# df=df.sample(n=n)

# df.to_csv('../data/raw/df_mini.csv')
df = pd.read_csv('../data/raw/df_mini.csv')



In [29]:
# a refaire en testant valeur de index
# rien en save, rien en read: ca marche pas

# save=False, rien en read: Ok c'est pareil. A noter sur un Nbook
# df.to_csv('../data/raw/df_test.csv', index=False)
df = pd.read_csv('../data/raw/df_test.csv')

C:\Users\ludov\AppData\Local\Temp\ipykernel_33468\1399070697.py:6: DtypeWarning: Columns (2,3,8,12,28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/df_test.csv')


In [57]:
# Inspection des données
co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES: FR/ALL 2021', save=False)
co2.styles.display_info(df, title='DONNEES BRUTES: FR/ALL 2021', save=False) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES: FR/ALL 2021', save=False)   


,Unnamed: 0,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,7184618,51944820,DE,IP-0000641-WBA-1,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,nan,E1*2007/46*1683*10,FMK,71BB,IHW500L0,MINI,CLUBMAN COOPER D,M1,M1,1,1510.000,1664.000,nan,129.000,2670.000,1563.000,1565.000,DIESEL,M,1995.000,110.000,nan,E24 28 29,nan,1.800,nan,nan,P,2021,2021-10-05,4.900,nan
1,1080904,53202066,PL,IP-MQB37WZ_A0_0583-VSS-1,VW-SAIC,SEAT,SEAT SA,nan,E9*2007/46*3167*11,KL,SFDTRDX0,FM6SFM6C90024BIVL1AAH,SEAT,SEAT LEON SP,M1,M1,1,1436.000,1585.000,nan,114.000,2672.000,1544.000,1514.000,DIESEL,M,1968.000,85.000,nan,e9 28 29,nan,1.900,nan,nan,P,2021,2021-08-17,3.600,nan
2,737056,55897558,NO,nan,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,nan,E1*2007/46*2017*05,G3K,6N71,IAW50700,BMW,330e xDrive,M1,M1,1,1965.000,nan,nan,38.000,2851.000,1589.000,1604.000,PETROL/ELECTRIC,P,1998.000,135.000,nan,nan,nan,nan,nan,nan,P,2021,2021-04-26,1.700,55.000
3,5373023,51798192,DE,IP-0000337-WBA-1,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,nan,E1*2007/46*1676*12,F1X,71AB,IAW50000,BMW,X1 XDRIVE 25E,M1,M1,1,1820.000,1944.000,43.000,43.000,2670.000,1563.000,1565.000,PETROL/ELECTRIC,P,1499.000,92.000,153.000,nan,nan,nan,nan,nan,P,2021,2021-06-24,7.500,nan
4,3292053,45055551,RO,IP-MEB31AZ_A0_0962-TMB-1,VW-SAIC,SKODA,SKODA AUTO AS,nan,E8*2007/46*0416*01,NY,ACEBJAL1FX2,D0MN1AA,SKODA,ENYAQ 80,M1,M1,1,2075.000,2324.000,0.000,0.000,2770.000,1587.000,1566.000,ELECTRIC,E,nan,nan,180.000,nan,nan,nan,nan,nan,P,2021,nan,nan,501.000


<class 'pandas.io.formats.style.Styler'>


,Column,Non-Null Count,Dtype
0,Unnamed: 0,100 non-null,int64
1,ID,100 non-null,int64
2,Country,100 non-null,object
3,VFN,88 non-null,object
4,Mp,88 non-null,object
5,Mh,100 non-null,object
6,Man,100 non-null,object
7,MMS,0 non-null,float64
8,Tan,99 non-null,object
9,T,100 non-null,object


<class 'pandas.io.formats.style.Styler'>


,Info
0,"RangeIndex: 100 entries, 0 to 99"
1,Data columns (total 39 columns):
2,38 Electric range (km) 22 non-null float64
3,"dtypes: float64(17), int64(4), object(18)"
4,memory usage: 30.6+ KB


<class 'pandas.io.formats.style.Styler'>


,Colonne,Valeurs manquantes (%)
0,MMS,100.00%
1,Vf,100.00%
2,De,100.00%
3,Ernedc (g/km),100.00%
4,Electric range (km),78.00%
5,z (Wh/km),78.00%
6,Enedc (g/km),74.00%
7,Erwltp (g/km),67.00%
8,IT,66.00%
9,Fuel consumption,39.00%


<class 'pandas.io.formats.style.Styler'>


In [27]:
# test de data_preprocess entier

# countries2 = None
countries2 = ['FR', 'DE']
rem_fuel_consumption=True
rem_axlewidth=True
rem_engine_capacity=True

# test de data_preprocess 
df = co2.data.data_preprocess(df, countries2)

d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.rename(name_dict, axis=1, inplace=True)


In [67]:
df.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

In [58]:
# test de data_preprocess part 1
# Ca ca marche ave None (apres l'autre)

# countries2=['FR', 'DE']
countries2 = None


if countries2 is not None:
    df = df[df['Country'].isin(countries2)]

In [59]:
df = co2.data.convert_dtypes(df)

In [66]:
# marche pas, donc retest dessous
df = co2.data.rename_columns(df)

In [61]:

abbrev_list = [
    'ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va', 'Ve', 'Mk',
    'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)', 'Ewltp (g/km)', 'W (mm)',
    'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT',
    'Ernedc (g/km)', 'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
    'Fuel consumption ', 'Electric range (km)'
    ]

nom_colonne_list = [
    'ID', 'Country', 'VehicleFamilyIdentification', 'Pool', 'ManufacturerName', 'ManufNameOem',
    'ManufNameMS', 'TypeApprovalNumber', 'Type', 'Variant', 'Version', 'Make', 'CommercialName',
    'VehicleCategory', 'CategoryOf', 'TotalNewRegistrations', 'MassRunningOrder',
    'WltpTestMass', 'Co2EmissionsNedc', 'Co2EmissionsWltp',
    'BaseWheel', 'AxleWidthSteering', 'AxleWidthOther', 'FuelType', 'FuelMode',
    'EngineCapacity', 'EnginePower', 'ElectricConsumption',
    'InnovativeTechnology', 'InnovativeEmissionsReduction',
    'InnovativeEmissionsReductionWltp', 'DeviationFactor', 'VerificationFactor',
    'Status', 'RegistrationYear', 'RegistrationDate', 'FuelConsumption', 'ElectricRange'
    ]
    
name_dict = dict(zip(abbrev_list, nom_colonne_list))


In [68]:
name_dict
df = df.rename(name_dict, axis=1)


{'ID': 'ID',
 'Country': 'Country',
 'VFN': 'VehicleFamilyIdentification',
 'Mp': 'Pool',
 'Mh': 'ManufacturerName',
 'Man': 'ManufNameOem',
 'MMS': 'ManufNameMS',
 'Tan': 'TypeApprovalNumber',
 'T': 'Type',
 'Va': 'Variant',
 'Ve': 'Version',
 'Mk': 'Make',
 'Cn': 'CommercialName',
 'Ct': 'VehicleCategory',
 'Cr': 'CategoryOf',
 'r': 'TotalNewRegistrations',
 'm (kg)': 'MassRunningOrder',
 'Mt': 'WltpTestMass',
 'Enedc (g/km)': 'Co2EmissionsNedc',
 'Ewltp (g/km)': 'Co2EmissionsWltp',
 'W (mm)': 'BaseWheel',
 'At1 (mm)': 'AxleWidthSteering',
 'At2 (mm)': 'AxleWidthOther',
 'Ft': 'FuelType',
 'Fm': 'FuelMode',
 'ec (cm3)': 'EngineCapacity',
 'ep (KW)': 'EnginePower',
 'z (Wh/km)': 'ElectricConsumption',
 'IT': 'InnovativeTechnology',
 'Ernedc (g/km)': 'InnovativeEmissionsReduction',
 'Erwltp (g/km)': 'InnovativeEmissionsReductionWltp',
 'De': 'DeviationFactor',
 'Vf': 'VerificationFactor',
 'Status': 'Status',
 'year': 'RegistrationYear',
 'Date of registration': 'RegistrationDate',
 'F

In [65]:
df.head(3)

,Unnamed: 0,ID,Country,VehicleFamilyIdentification,Pool,ManufacturerName,ManufNameOem,ManufNameMS,TypeApprovalNumber,Type,...,InnovativeTechnology,InnovativeEmissionsReduction,InnovativeEmissionsReductionWltp,DeviationFactor,VerificationFactor,Status,RegistrationYear,RegistrationDate,FuelConsumption,ElectricRange
0,7184618,51944820,DE,IP-0000641-WBA-1,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,NaN,E1*2007/46*1683*10,FMK,...,E24 28 29,NaN,1.8,NaN,NaN,P,2021,2021-10-05,4.9,NaN
1,1080904,53202064,PL,IP-MQB37WZ_A0_0583-VSS-1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*11,KL,...,e9 28 29,NaN,1.9,NaN,NaN,P,2021,2021-08-17,3.6,NaN
2,737056,55897560,NO,NaN,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,NaN,E1*2007/46*2017*05,G3K,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-04-26,1.7,55.0


In [22]:
# test de rename colomns
df = pd.read_csv('../data/raw/df_test.csv')

C:\Users\ludov\AppData\Local\Temp\ipykernel_33468\3914182695.py:2: DtypeWarning: Columns (2,3,8,12,28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/df_test.csv')


In [ ]:
def rename_columns(df):
    abbrev_list = [
        'ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va', 'Ve', 'Mk',
        'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)', 'Ewltp (g/km)', 'W (mm)',
        'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT',
        'Ernedc (g/km)', 'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
        'Fuel consumption ', 'Electric range (km)'
        ]

    nom_colonne_list = [
        'ID', 'Country', 'VehicleFamilyIdentification', 'Pool', 'ManufacturerName', 'ManufNameOem',
        'ManufNameMS', 'TypeApprovalNumber', 'Type', 'Variant', 'Version', 'Make', 'CommercialName',
        'VehicleCategory', 'CategoryOf', 'TotalNewRegistrations', 'MassRunningOrder',
        'WltpTestMass', 'Co2EmissionsNedc', 'Co2EmissionsWltp',
        'BaseWheel', 'AxleWidthSteering', 'AxleWidthOther', 'FuelType', 'FuelMode',
        'EngineCapacity', 'EnginePower', 'ElectricConsumption',
        'InnovativeTechnology', 'InnovativeEmissionsReduction',
        'InnovativeEmissionsReductionWltp', 'DeviationFactor', 'VerificationFactor',
        'Status', 'RegistrationYear', 'RegistrationDate', 'FuelConsumption', 'ElectricRange'
        ]
    
    name_dict = dict(zip(abbrev_list, nom_colonne_list))
    df = df.rename(name_dict, axis=1, inplace=True)
    return df

df = rename_columns(df)

In [15]:
# décomposition de ml_preprocess part 1
# On s'arrete juste avant  df = df['ElectricRange'].fillna(0)

# countries = None
countries = ['FR', 'DE']
rem_fuel_consumption=True
rem_axlewidth=True
rem_engine_capacity=True

rows_t0 = len(df)
df = co2.data.data_preprocess(df, countries)

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,SBDTTCX0,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,DLAAX0AC4,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,YLDC1FX,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,DLAAX0AC4,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,R,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [10]:
# Inspection des données
co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES: FR/ALL 2021', save=False)
co2.styles.display_info(df, title='DONNEES BRUTES: FR/ALL 2021', save=False) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES: FR/ALL 2021', save=False)   

# Nettoyage des données
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)


# discretize electricrange only for classification
# df = co2.data.discretize_electricrange(df, to_dummies=False)

# Inspection des features
co2.styles.display_info(df, title='DONNEES NETTOYEES: FR/ALL 2021', save=False) 
co2.styles.display_describe(df, title='STATISTIQUES DESCRIPTIVES: FR/ALL 2021', save=False)

co2.viz.plot_correlation_heatmap(df, save=False, interactive=False, title=': FR/ALL 2021')
co2.viz.plot_feature_distributions(df, interactive=False, save=TFalse, title=': FR/ALL 2021')
co2.viz.plot_qqplots(df, interactive=False, save=False, title=': FR/ALL 2021')

# One-hot encoding des variables catégorielles
df = co2.data.dummify_all_categoricals(df, should_discretize_electricrange=False)

# Sauvegarde du jeu de données prêt à l'emploi pour la régression
co2.data.save_processed_data(df, classification=False, pickle=False)

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,nan,E9*2007/46*3167*07,KL,SBDTTCX0,FD7SFD7GC0024BIVL1BAF,SEAT,LEON,M1,M1,1,1448.000,nan,104.000,118.000,2669.000,1550.000,1520.000,DIESEL,M,1968.000,110.000,nan,e9 28 29,nan,1.700,nan,nan,P,2021,2021-01-11,4.500,nan
1,295040,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*17,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNMI,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 28 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,nan,E13*2007/46*2188*03,DFK,YLDC1FX,577CJ6AP1AD,FORD,KUGA,M1,M1,1,1687.000,nan,113.000,135.000,2710.000,1596.000,1580.000,DIESEL,M,1995.000,110.000,nan,nan,nan,nan,nan,nan,P,2021,2021-03-24,nan,nan
3,295042,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*18,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNI0,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
4,295043,ES,nan,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,nan,E2*2007/46*0597*19,Z,R,HNSK-C1P600,OPEL,GRANDLAND,M1,M1,1,1395.000,nan,114.000,140.000,2675.000,1595.000,1610.000,PETROL,M,1199.000,96.000,nan,nan,nan,2.100,nan,nan,P,2021,2021-02-25,6.200,nan


<class 'pandas.io.formats.style.Styler'>


,Column,Non-Null Count
0,ID,int64
1,Country,object
2,VFN,object
3,Mp,object
4,Mh,object
5,Man,object
6,MMS,float64
7,Tan,object
8,T,object
9,Va,object


<class 'pandas.io.formats.style.Styler'>


,Info
0,"RangeIndex: 9920108 entries, 0 to 9920107"
1,Data columns (total 38 columns):
2,37 Electric range (km) float64
3,"dtypes: float64(17), int64(3), object(18)"
4,memory usage: 2.8+ GB


<class 'pandas.io.formats.style.Styler'>


,Colonne,Valeurs manquantes (%)
0,Vf,100.00%
1,Ernedc (g/km),100.00%
2,De,100.00%
3,MMS,100.00%
4,Electric range (km),85.24%
5,z (Wh/km),83.63%
6,Enedc (g/km),75.63%
7,Erwltp (g/km),56.70%
8,IT,54.13%
9,Fuel consumption,40.73%


<class 'pandas.io.formats.style.Styler'>


d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.rename(name_dict, axis=1, inplace=True)


AttributeError: 'NoneType' object has no attribute 'drop'

d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.rename(name_dict, axis=1, inplace=True)


In [13]:
df

In [ ]:
df = co2.data.drop_irrelevant_columns(df)

In [6]:
to_drop = [
            'VehicleFamilyIdentification', 'ManufNameMS', 'TypeApprovalNumber', 
            'Type', 'Variant', 'Version', 'VehicleCategory',
           'TotalNewRegistrations', 'Co2EmissionsNedc', 'AxleWidthOther', 'Status',
           'InnovativeEmissionsReduction', 'DeviationFactor', 'VerificationFactor', 'Status',
           'RegistrationYear']
    
df1 = df.drop(columns=[col for col in to_drop if col in df.columns])

In [7]:
df1

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,SBDTTCX0,...,NaN,e9 28 29,NaN,1.7,NaN,NaN,2021,2021-01-11,4.5,NaN
1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,DLAAX0AC4,...,NaN,E13 28 29,NaN,NaN,NaN,NaN,2021,2021-05-18,6.1,NaN
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,YLDC1FX,...,NaN,NaN,NaN,NaN,NaN,NaN,2021,2021-03-24,NaN,NaN
3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,DLAAX0AC4,...,NaN,E13 29,NaN,NaN,NaN,NaN,2021,2021-05-18,6.1,NaN
4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,R,...,NaN,NaN,NaN,2.1,NaN,NaN,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,50547899,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,NaN,E9 29 37,NaN,1.7,NaN,NaN,2021,2021-11-24,6.4,NaN
9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,NaN,E9 29 37,NaN,1.7,NaN,NaN,2021,2021-10-21,6.4,NaN
9920105,50547901,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,NaN,E9 29 37,NaN,1.6,NaN,NaN,2021,2021-11-25,6.4,NaN
9920106,50547902,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,DHE2,...,NaN,E9 29 37,NaN,1.9,NaN,NaN,2021,2021-11-15,6.2,NaN


In [49]:
# Inspection des données
co2.styles.displayer(df, title='APERCU DU JEU DE DONNEES: FR/ALL 2021', save=True)
co2.styles.display_info(df, title='DONNEES BRUTES: FR/ALL 2021', save=True) 
co2.styles.display_na(df, title='PART DE VALEURS MANQUANTES DANS LE JEU DE DONNEES: FR/ALL 2021', save=True)   


,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,nan,E9*2007/46*3167*07,KL,SBDTTCX0,FD7SFD7GC0024BIVL1BAF,SEAT,LEON,M1,M1,1,1448.000,nan,104.000,118.000,2669.000,1550.000,1520.000,DIESEL,M,1968.000,110.000,nan,e9 28 29,nan,1.700,nan,nan,P,2021,2021-01-11,4.500,nan
1,295040,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*17,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNMI,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 28 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,nan,E13*2007/46*2188*03,DFK,YLDC1FX,577CJ6AP1AD,FORD,KUGA,M1,M1,1,1687.000,nan,113.000,135.000,2710.000,1596.000,1580.000,DIESEL,M,1995.000,110.000,nan,nan,nan,nan,nan,nan,P,2021,2021-03-24,nan,nan
3,295042,ES,nan,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,nan,E13*2007/46*1845*18,A1,DLAAX0AC4,FM6FM6AJ0254BI1AANNI0,VOLKSWAGEN,T-ROC,M1,M1,1,1294.000,nan,nan,137.000,2590.000,1546.000,1540.000,PETROL,M,999.000,81.000,nan,E13 29,nan,nan,nan,nan,P,2021,2021-05-18,6.100,nan
4,295043,ES,nan,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,nan,E2*2007/46*0597*19,Z,R,HNSK-C1P600,OPEL,GRANDLAND,M1,M1,1,1395.000,nan,114.000,140.000,2675.000,1595.000,1610.000,PETROL,M,1199.000,96.000,nan,nan,nan,2.100,nan,nan,P,2021,2021-02-25,6.200,nan


<class 'pandas.io.formats.style.Styler'>


,Column,Non-Null Count
0,ID,int64
1,Country,object
2,VFN,object
3,Mp,object
4,Mh,object
5,Man,object
6,MMS,float64
7,Tan,object
8,T,object
9,Va,object


<class 'pandas.io.formats.style.Styler'>


,Info
0,"RangeIndex: 9920108 entries, 0 to 9920107"
1,Data columns (total 38 columns):
2,37 Electric range (km) float64
3,"dtypes: float64(17), int64(3), object(18)"
4,memory usage: 2.8+ GB


<class 'pandas.io.formats.style.Styler'>


,Colonne,Valeurs manquantes (%)
0,Vf,100.00%
1,Ernedc (g/km),100.00%
2,De,100.00%
3,MMS,100.00%
4,Electric range (km),85.24%
5,z (Wh/km),83.63%
6,Enedc (g/km),75.63%
7,Erwltp (g/km),56.70%
8,IT,54.13%
9,Fuel consumption,40.73%


<class 'pandas.io.formats.style.Styler'>


In [7]:
countries=['FR', 'DE']
rem_fuel_consumption=True
rem_axlewidth=True
rem_engine_capacity=True

In [11]:
# ca marche pas apres df = df['ElectricRange'].fillna(0)
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)

d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(name_dict, axis=1, inplace=True)


AttributeError: 'Series' object has no attribute 'columns'

In [51]:
# décomposition de ml_preprocess part 1
# On s'arrete juste avant  df = df['ElectricRange'].fillna(0)

# countries = None
countries = ['FR', 'DE']
rem_fuel_consumption=True
rem_axlewidth=True
rem_engine_capacity=True

rows_t0 = len(df)
df = co2.data.data_preprocess(df, countries)
df = co2.data.drop_irrelevant_columns(df)
df = co2.data.remove_columns(df, axlewidth=rem_axlewidth, engine_capacity=rem_engine_capacity, fuel_consumption=rem_fuel_consumption)  
df = co2.data.standardize_innovtech(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)
display(df.info())


d:\0 PERSO\00 LUDO\Google Drive\0 LUDO\Projets\Cours_info\05 Projet Dts\github\car-co2-prediction_pre-release\notebooks\auto_co2\data.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(name_dict, axis=1, inplace=True)


-0.5657294255264156
<class 'pandas.core.frame.DataFrame'>
Index: 4308011 entries, 50000 to 9920107
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Pool                  object 
 1   MassRunningOrder      float64
 2   Co2EmissionsWltp      float64
 3   BaseWheel             float64
 4   FuelType              object 
 5   EnginePower           float64
 6   InnovativeTechnology  int32  
 7   ElectricRange         float64
dtypes: float64(5), int32(1), object(2)
memory usage: 279.4+ MB


None

### Pb avec country = None


In [9]:
# Ca ca marche ave None (apres l'autre)

# countries=['FR', 'DE']
countries = None
print(countries)

if countries is not None:
    df = df[df['Country'].isin(countries)]


None


In [37]:
df

,Pool,MassRunningOrder,Co2EmissionsWltp,BaseWheel,FuelType,EnginePower,InnovativeTechnology,ElectricRange
50000,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,NaN
50001,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,NaN
50002,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,NaN
50003,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,NaN
50004,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,NaN
...,...,...,...,...,...,...,...,...
9920103,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,NaN
9920104,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,NaN
9920105,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,NaN
9920106,RENAULT-NISSAN-MITSUBISHI,1264.0,140.0,2673.0,PETROL,67.0,1,NaN


In [53]:
df

,Pool,MassRunningOrder,Co2EmissionsWltp,BaseWheel,FuelType,EnginePower,InnovativeTechnology,ElectricRange
50000,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,0.0
50001,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,0.0
50002,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,0.0
50003,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,0.0
50004,STELLANTIS,1598.0,0.0,2558.0,ELECTRIC,57.0,0,0.0
...,...,...,...,...,...,...,...,...
9920103,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,0.0
9920104,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,0.0
9920105,RENAULT-NISSAN-MITSUBISHI,1371.0,145.0,2673.0,LPG,67.0,1,0.0
9920106,RENAULT-NISSAN-MITSUBISHI,1264.0,140.0,2673.0,PETROL,67.0,1,0.0


In [52]:
rows_t0 = len(df)
df['ElectricRange'] = df['ElectricRange'].fillna(0)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [44]:
df.head(3)
df.isna().mean().sort_values(ascending=False)

Pool                    0.022665
EnginePower             0.007501
BaseWheel               0.003935
Co2EmissionsWltp        0.001699
MassRunningOrder        0.000001
FuelType                0.000000
InnovativeTechnology    0.000000
ElectricRange           0.000000
dtype: float64

In [128]:
# df = convert_dtypes(df)
# df = rename_columns(df)

floats = df.select_dtypes(include=['float']).columns
df.loc[:, floats] = df.loc[:, floats].astype('float32')
ints = df.select_dtypes(include=['int']).columns
df.loc[:, ints] = df.loc[:, ints].astype('float32')
objs = df.select_dtypes(include=['object']).columns
df.loc[:, objs] = df.loc[:, objs].astype('category')

In [133]:
abbrev_list = [
        'ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va', 'Ve', 'Mk',
        'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)', 'Ewltp (g/km)', 'W (mm)',
        'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT',
        'Ernedc (g/km)', 'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
        'Fuel consumption ', 'Electric range (km)'
        ]
nom_colonne_list = [
        'ID', 'Country', 'VehicleFamilyIdentification', 'Pool', 'ManufacturerName', 'ManufNameOem',
        'ManufNameMS', 'TypeApprovalNumber', 'Type', 'Variant', 'Version', 'Make', 'CommercialName',
        'VehicleCategory', 'CategoryOf', 'TotalNewRegistrations', 'MassRunningOrder',
        'WltpTestMass', 'Co2EmissionsNedc', 'Co2EmissionsWltp',
        'BaseWheel', 'AxleWidthSteering', 'AxleWidthOther', 'FuelType', 'FuelMode',
        'EngineCapacity', 'EnginePower', 'ElectricConsumption',
        'InnovativeTechnology', 'InnovativeEmissionsReduction',
        'InnovativeEmissionsReductionWltp', 'DeviationFactor', 'VerificationFactor',
        'Status', 'RegistrationYear', 'RegistrationDate', 'FuelConsumption', 'ElectricRange'
        ]
name_dict = dict(zip(abbrev_list, nom_colonne_list))
df.rename(name_dict, axis=1, inplace=True)

In [134]:
df

,Unnamed: 0,ID,Country,VehicleFamilyIdentification,Pool,ManufacturerName,ManufNameOem,ManufNameMS,TypeApprovalNumber,Type,...,InnovativeTechnology,InnovativeEmissionsReduction,InnovativeEmissionsReductionWltp,DeviationFactor,VerificationFactor,Status,RegistrationYear,RegistrationDate,FuelConsumption,ElectricRange
0,0,295039,ES,IP MQB37WZ_A1_0589 VSS 1,VW-SAIC,SEAT,SEAT SA,NaN,E9*2007/46*3167*07,KL,...,e9 28 29,NaN,1.7,NaN,NaN,P,2021,2021-01-11,4.5,NaN
1,1,295040,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*17,A1,...,E13 28 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
2,2,295041,ES,IP-C482_2019_00002-WF0-1,FORD,FORD WERKE GMBH,FORD WERKE GMBH,NaN,E13*2007/46*2188*03,DFK,...,NaN,NaN,NaN,NaN,NaN,P,2021,2021-03-24,NaN,NaN
3,3,295042,ES,NaN,VW-SAIC,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*18,A1,...,E13 29,NaN,NaN,NaN,NaN,P,2021,2021-05-18,6.1,NaN
4,4,295043,ES,NaN,STELLANTIS,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,NaN,E2*2007/46*0597*19,Z,...,NaN,NaN,2.1,NaN,NaN,P,2021,2021-02-25,6.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920103,9920103,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-11-24,6.4,NaN
9920104,9920104,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.7,NaN,NaN,P,2021,2021-10-21,6.4,NaN
9920105,9920105,50547900,DE,IP-JDA1MTGJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.6,NaN,NaN,P,2021,2021-11-25,6.4,NaN
9920106,9920106,50547904,DE,IP-JDA1M6PJT4B_000-VF1-1,RENAULT-NISSAN-MITSUBISHI,RENAULT,RENAULT SAS,NaN,E2*2001/116*0323*65,SR,...,E9 29 37,NaN,1.9,NaN,NaN,P,2021,2021-11-15,6.2,NaN


In [107]:
rows_t0 = len(df)
df = co2.data.drop_irrelevant_columns(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [108]:
rows_t0 = len(df)
df = co2.data.remove_columns(df, axlewidth=rem_axlewidth, engine_capacity=rem_engine_capacity, fuel_consumption=rem_fuel_consumption) 
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [109]:
rows_t0 = len(df)
df = co2.data.standardize_innovtech(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [110]:
rows_t0 = len(df)
df = co2.data.drop_residual_incomplete_rows(df)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)


Incomplete rows dropped:130124
-0.030205122503169098


In [113]:
len(df)

4177887

In [112]:

rows_t0 = len(df)
df = df['ElectricRange'].fillna(0)
rows_t1 = len(df)
print((rows_t1-rows_t0)/rows_t0)

0.0


In [ ]:
def ml_preprocess(df, countries=None, 
                     rem_fuel_consumption=True, 
                     rem_axlewidth=True, 
                     rem_engine_capacity=True):
    
    rows_t0 = len(df)
    
    df = data_preprocess(df, countries)
    df = drop_irrelevant_columns(df)
    df = remove_columns(df, axlewidth=rem_axlewidth, engine_capacity=rem_engine_capacity, fuel_consumption=rem_fuel_consumption)  
    df = standardize_innovtech(df)
    df = drop_residual_incomplete_rows(df)
    df = df['ElectricRange'].fillna(0, inplace=True)

    rows_t1 = len(df)
    print(f"TOTAL NUMBER OF ROWS DROPPED:{rows_t0 - rows_t1}")
    
    return df

In [ ]:

# discretize electricrange only for classification
# df = co2.data.discretize_electricrange(df, to_dummies=False)

# Inspection des features
co2.styles.display_info(df, title='DONNEES NETTOYEES: FR/ALL 2021', save=True) 
co2.styles.display_describe(df, title='STATISTIQUES DESCRIPTIVES: FR/ALL 2021', save=True)

co2.viz.plot_correlation_heatmap(df, save=True, interactive=False, title=': FR/ALL 2021')
co2.viz.plot_feature_distributions(df, interactive=False, save=True, title=': FR/ALL 2021')
co2.viz.plot_qqplots(df, interactive=False, save=True, title=': FR/ALL 2021')


In [ ]:

# One-hot encoding des variables catégorielles
df = co2.data.dummify_all_categoricals(df, should_discretize_electricrange=False)


In [ ]:

# Sauvegarde du jeu de données prêt à l'emploi pour la régression
co2.data.save_processed_data(df, classification=False, pickle=False)

In [49]:
df = co2.data.ml_preprocess(df,
                               countries=['FR', 'DE'],
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)

KeyError: 'Country'